In [2]:
!pip install selenium

  Using cached selenium-4.13.0-py3-none-any.whl (9.5 MB)
  Using cached trio-0.22.2-py3-none-any.whl (400 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
  Using cached exceptiongroup-1.1.3-py3-none-any.whl (14 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [98]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests
import re
import csv

def extract_google_reviews(movie_name, file_name):
    # Initialize the Chrome web driver
    driver = webdriver.Chrome()

    # Search for the movie on Google
    driver.get("https://www.imdb.com/")
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(f"{movie_name}")
    search_box.send_keys(Keys.RETURN)

    # Click on the "All reviews" button to see more reviews
    home_url = driver.find_element(By.CLASS_NAME, "ipc-metadata-list-summary-item__t").get_attribute("href")
    home_url = home_url.split("?")[0]
    driver.get(f"{home_url}reviews?ref_=tt_urv")

    cnt = 0
    while cnt < 7:
        try:
            button = driver.find_element(By.CLASS_NAME, "ipl-load-more__button")
            button.click()
            time.sleep(4)
            cnt += 1
        except:
            break
    
    rows = [] 

    complete_reviews = driver.find_elements(By.CLASS_NAME, "lister-item-content")
    for complete_review in complete_reviews:
        date = complete_review.find_element(By.CLASS_NAME, "review-date").text
        title = complete_review.find_element(By.CLASS_NAME, "title").text
        try:
            rating = complete_review.find_element(By.CLASS_NAME, "rating-other-user-rating").text[:-3]
        except:
            rating = 0
        try:
            review = complete_review.find_element(By.CLASS_NAME, "content").text
        except:
            review = ""

        if review == "":
            continue
        
        review = review.replace("\n", "")
        review = review.replace("\\'", "'")
        pattern = "\d+ out of \d+ found this helpful. Was this review helpful"
        removed_text = re.sub(pattern, "", review)
        pattern = "Sign in to vote."
        removed_text = re.sub(pattern, "", removed_text)
        pattern = "Permalink"
        removed_text = re.sub(pattern, "", removed_text)
        review = str(removed_text)
        
        rows.append([movie_name, date, title, review, rating])
    print(len(rows))
    with open(file_name, 'a', encoding="utf-8") as f:
        csvwriter = csv.writer(f, delimiter='~')
        csvwriter.writerows(rows)
    driver.quit()

In [90]:
fields = ['Movie Name', 'Date', 'Title', 'Review', 'Rating']
with open('reviews.csv', 'w', encoding="utf-8") as f:
    csvwriter = csv.writer(f, delimiter='~')
    csvwriter.writerow(fields)

In [94]:
with open('movies.txt') as f1:
    movies = f1.readlines()
f1.close()

# extract_google_reviews("Star Wars: Episode V - The Empire Strikes Back", "reviews.csv")
for movie in movies:
    movie = movie.replace("\n", "")
    movie_name = str(movie)
    extract_google_reviews(movie_name)

151


In [99]:
fields = ['Show Name', 'Date', 'Title', 'Review', 'Rating']
with open('reviews1.csv', 'w', encoding="utf-8") as f:
    csvwriter = csv.writer(f, delimiter='~')
    csvwriter.writerow(fields)

In [100]:
with open('tvshows.txt') as f1:
    shows = f1.readlines()
f1.close()

for show in shows:
    show = show.replace("\n", "")
    show_name = str(show)
    extract_google_reviews(show_name, "reviews1.csv")

42
116
94
90
23
112
24
49
176
134
111
193


KeyboardInterrupt: 

###### 